In [182]:
import matplotlib
matplotlib.use('Agg')

import sys
sys.settrace
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pylab import *
import struct
import array
import os
import glob
import h5py
from scipy.interpolate import griddata
from scipy import integrate

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
#plt.rcParams.update({
#    "text.usetex": True,
#    "font.family": "serif",
#    "font.serif": ["Palatino"],
#})

plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
plt.rcParams['font.size'] = '20'

from importlib import reload

In [84]:
import athena_read

In [178]:
def FitBlackBody(nu_t):
  integral = 0.0;
  nu_2 = nu_t * nu_t;
  nu_3 = nu_t * nu_2;
  if nu_t < 1.5:
    integral = 0.051329911273422 * nu_3 -0.019248716727533 * nu_t * nu_3 + 0.002566495563671 * nu_2 * nu_3;
  elif nu_t < 18.6:
    exp_nu = exp(-nu_t);
    integral = -0.156915538762850 * exp_nu * (nu_3 + 2.69 * nu_2 + 6.714 * nu_t) + 1.000009331428801*(1- exp_nu);
  else:
    integral = 1.0 - 192.1*exp(-0.9014*nu_t)

  return integral;
def BlackBody(nu_t):
  spec=(15/np.pi**4.0)*nu_t**3/(exp(nu_t)-1)

  return spec;

In [193]:
def PlotProfile(datax, datay, xmin, xmax, ymin, ymax,  ylabel, label1, filename, xlabel='$r/r_g$', logscale=0, 
                xlogscale=0,datay1_2=None, datay1_3=None, datax2=None, datay2=None, datay2_2=None, datay2_3=None, 
                datax3=None, datay3=None, datay3_2=None, datay3_3=None, datax4=None, datay4=None, 
                datax5=None, datay5=None, label2='', label3='', label4='', label5='',title=None,leg_loc=None):
    plots, axes = plt.subplots(figsize=(9,11),dpi=300)
    plt.xlabel(xlabel, size = 30)
    plt.ylabel(ylabel, size = 30)
    plt.subplots_adjust(left=0.16,right=0.88,top=0.9,bottom=0.1)
    plt.ylim([ymin,ymax])
    plt.xlim([xmin,xmax])
    if logscale > 0:
      axes.set_yscale('log')
    if xlogscale > 0:
      axes.set_xscale('log')
    if title is not None:
      plt.title(title,size=20)

    plt.plot(datax,datay,color='black',marker=' ',fillstyle='none',markersize=8,label=label1,linewidth=3.0)
    if datay1_2 is not None:
      plt.scatter(datax,datay1_2,color='red',marker='o',s=40)
    if datay1_3 is not None:
      plt.plot(datax,datay1_3,color='black',linestyle='dashed',linewidth=4.0)
    if datay2 is not None:
      plt.plot(datax2,datay2,color='red',label=label2,linewidth=2.0,alpha=1.0,marker='o',
               linestyle='dashed',markersize=10,fillstyle='none')
    if datay2_2 is not None:
      plt.plot(datax2,datay2_2,color='black',linestyle='dashed',linewidth=2.0,alpha=1.0)
    if datay2_3 is not None:
      plt.plot(datax2,datay2_3,color='red',linestyle='dashed',linewidth=4.0)
    if datay3 is not None:
      plt.plot(datax3,datay3,color='green',label=label3,marker='s',fillstyle='none',markersize=8,linewidth=3.0)
    if datay3_2 is not None:
      plt.plot(datax3,datay3_2,color='black',linestyle='dashed',linewidth=2.0,alpha=1.0)
    if datay3_3 is not None:
      plt.plot(datax3,datay3_3,color='green',linestyle='dashed',linewidth=4.0)
    if datay4 is not None:
      plt.plot(datax4,datay4,color='red',linestyle='dotted',label=label4,linewidth=3.0)
    if datay5 is not None:
      plt.plot(datax5,datay5,color=tableau20[0],label=label5,linewidth=2.0) 
    if leg_loc is not None:
      plt.legend(loc="best",bbox_to_anchor=leg_loc,frameon=False)
    axes.set_aspect('auto')
#    axes.yaxis.set_tick_params(labelsize=25)
#    axes.xaxis.set_tick_params(labelsize=25)
    plt.savefig(filename)
    plt.close(plots)


In [152]:
files=sorted(glob.glob('Data/thermal*athdf'))
num_file=len(files)

In [153]:
#for filename in files:
ang_file = open('Data/Rad_angles.txt', 'r')
Lines = ang_file.readlines()
location=Lines.index('fre   spec\n')
tot_line=size(Lines)
nfreq=tot_line-location-1
nu_grid=np.zeros(nfreq)
histories=np.zeros((num_file,5))
for i in range(nfreq):
    line=Lines[i+location+1].split(' ')[0]
    nu_grid[i]=float(line)

nu_center=(nu_grid[:-1]+nu_grid[1:])/2
nu_center=np.append(nu_center,nu_grid[nfreq-1])

In [154]:
Prat=1
Crat=10
count=0
filename=files[num_file-1]

with h5py.File(filename, 'r') as f:
  attributes = f.attrs.items()
  attrs = dict(attributes)
  level = f.attrs['MaxLevel']
  time = f.attrs['Time']
  subsample = False

data = athena_read.athdf(filename, level=level, subsample=subsample)

Er_spec=np.zeros(nfreq)
for i in range(nfreq):
    varname='Er_'+str(i)
    Er_spec[i]=np.average(data[varname])
    
tgas=np.average(data['press']/data['rho'])
blackbody=np.zeros(nfreq)
for i in range(nfreq-1):
    kt_right=nu_grid[i+1]/tgas
    kt_left=nu_grid[i]/tgas
    blackbody[i]=tgas**4.0*(FitBlackBody(kt_right)-FitBlackBody(kt_left))
blackbody[nfreq-1]=tgas**4.0*(1-FitBlackBody(nu_grid[nfreq-1]/tgas))

In [196]:
ylabel='$\\rm Integrated\\ Spectrum \int_{\\nu_1}^{\\nu_2} E_r(\\nu) d\\nu$'
filename='integrated_spectrum.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_center,Er_spec,2.e-3, 50, 1.e-8, 100, ylabel, None, filename, xlabel, logscale=1,
            datay1_2=blackbody,xlogscale=1)

In [186]:
spec_int=np.zeros(nfreq-1)
for i in range(nfreq-1):
    spec_int[i]=Er_spec[i]/(tgas**4.0*(nu_grid[i+1]-nu_grid[i]))
nu_center2=(nu_grid[:-1]+nu_grid[1:])/2
nu_plot=np.logspace(-3., 2., num=1000)
bd_spec=BlackBody(nu_plot/tgas)/tgas

In [195]:
ylabel='$\\rm  E_r(\\nu)/a_rT_g^4 $'
filename='thermal_spectrum.pdf'
xlabel='$\\nu$'

#print(histories[:,0])
PlotProfile(nu_plot,bd_spec,2.e-3, 50, 1.e-6, 1, ylabel, None, filename, xlabel, logscale=1,
            xlogscale=1,datax2=nu_center2,datay2=spec_int)

In [183]:
integrate.quad(BlackBody, 0, 40)

(0.9999999999999549, 1.320824794960577e-09)

In [83]:
print(1-FitBlackBody(18.6))
print(1-FitBlackBody(18.61))
print(1-FitBlackBody(44))

1.0049135669154552e-05
9.958959793499034e-06
1.1102230246251565e-15
